In [1]:
import numpy as np
import trimesh

# checking that decimated then screened poisson can be skeletonized

In [ ]:
import os
directory = "./temp"
if not os.path.exists(directory):
    os.makedirs(directory)


input_mesh = "96631955273149705_decimated_poisson.off"
output_mesh = "temp/96631955273149705_decimated_poisson_largest_piece.off"

new_mesh = trimesh.load_mesh(input_mesh)
mesh_splits = new_mesh.split(only_watertight=True)
mesh_lengths = np.array([len(split.faces) for split in mesh_splits])
largest_index = np.where(mesh_lengths == np.max(mesh_lengths))
largest_mesh = mesh_splits[largest_index][0]


largest_mesh.export(output_mesh)
print("done exporting")

In [2]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,
                                vertices=[], 
                                triangles=[],
                                folder="pymesh_NEURONS"):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    file_loc = pathlib.Path.cwd() / folder
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

In [ ]:
decimated_mesh = trimesh.load_mesh("96631955273149705_decimated.off")

files = write_Whole_Neuron_Off_file("96631955273149705_decimated_bac",
                                vertices=decimated_mesh.vertices, 
                                triangles=decimated_mesh.faces,
                                folder="temp")


In [ ]:
files[0]

# -------------------visualizing the skeletonization------------------------

# Example ipyvolume visualization

In [ ]:
import ipyvolume as ipv

In [ ]:
# # Example ipyvolume

# s = 1/2**2
# # 4 vertices for the tetrahedron
# x = np.array([1.,  -1, 0,  0])
# y = np.array([0,   0, 1., -1])
# z = np.array([-s, -s, s,  s])
# # and 4 surfaces (triangles), where the number refer to the vertex index
# triangles = [(0, 1, 2), (0, 1, 3), (0, 2, 3), (1,3,2)]

# ipv.figure()
# # we draw the tetrahedron
# mesh = ipv.plot_trisurf(x, y, z, triangles=triangles, color='orange')
# # and also mark the vertices
# ipv.scatter(x, y, z, marker='sphere', color='blue')
# ipv.xyzlim(-2, 2)
# ipv.show()

# Visualizing A whole neuron

In [ ]:
ipv.figure()
# we draw the tetrahedron
mesh = ipv.plot_trisurf(largest_mesh.vertices[:,0], largest_mesh.vertices[:,1], largest_mesh.vertices[:,2], triangles=largest_mesh.faces, color='orange')
# and also mark the vertices

volume_maxs = np.max(largest_mesh.vertices,axis=0)
volume_mins = np.min(largest_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

# Visualizing the 3D skeleton

In [ ]:
#read in the skeleton files into an array
def read_skeleton_revised(file_path):
    with open(file_path) as f:
        bones = np.array([])
        for line in f.readlines():
            #print(line)
            line = (np.array(line.split()[1:], float).reshape(-1, 3))
            #print(line[:-1])
            #print(line[1:])

            #print(bones.size)
            if bones.size <= 0:
                bones = np.stack((line[:-1],line[1:]),axis=1)
            else:
                bones = np.vstack((bones,(np.stack((line[:-1],line[1:]),axis=1))))
            #print(bones)


    return np.array(bones).astype(float)
path_and_filename = "temp/96631955273149705_decimated_poisson_largest_piece"
bone_array = read_skeleton_revised(path_and_filename+"_skeleton.cgal")

In [ ]:
unique_skeleton_verts = bone_array.reshape(-1,3)
edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)

In [ ]:
ipv.figure()
# we draw the tetrahedron

mesh = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

volume_maxs = np.max(unique_skeleton_verts,axis=0)
volume_mins = np.min(unique_skeleton_verts,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

# Plot Neuron and skeleton at the same time

In [ ]:
ipv.figure()
# we draw the tetrahedron
mesh = ipv.plot_trisurf(largest_mesh.vertices[:,0], largest_mesh.vertices[:,1], largest_mesh.vertices[:,2], triangles=largest_mesh.faces, color='orange')
mesh.color = [0., 1., 0., 0.5]
mesh.material.transparent = True
# and also mark the vertices
mesh2 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

volume_maxs = np.max(largest_mesh.vertices,axis=0)
volume_mins = np.min(largest_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

# Getting the Meshlab decimation script working in python

In [3]:
def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    command_to_run = 'xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + script_command
    #command_to_run = 'meshlabserver ' + script_command
    
    print(command_to_run)
    subprocess_result = subprocess.run(command_to_run,shell=True)
    
    return subprocess_result

import os, contextlib
import pathlib
import subprocess
def meshlab_fix_manifold_path_specific_mls(input_path_and_filename,
                                           output_path_and_filename="",
                                           segment_id=-1,meshlab_script=""):
    #fix the path if it comes with the extension
    if input_path_and_filename[-4:] == ".off":
        path_and_filename = input_path_and_filename[:-4]
        input_mesh = input_path_and_filename
    else:
        raise Exception("Not passed off file")
    
    
    if output_path_and_filename == "":
        output_mesh = path_and_filename+"_mls.off"
    else:
        output_mesh = output_path_and_filename
    
    if meshlab_script == "":
        meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    #print("meshlab_script = " + str(meshlab_script))
    #print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(segment_id) + 
                         ' did not fix the manifold edges')
    
    return output_mesh


# How to do the screened poisson with meshlab

In [ ]:
script_name = "poisson_working_meshlab.mls"

input_file_base = "/notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/Decimation/temp/neuron_96631955273149705_decimated_bac"
output_file = input_file_base + "_poisson"
meshlab_script_path_and_name = str(pathlib.Path.cwd()) + "/" + script_name


meshlab_fix_manifold_path_specific_mls(input_path_and_filename=input_file_base + ".off",
                                                   output_path_and_filename=output_file + ".off",
                                                   meshlab_script=meshlab_script_path_and_name)

# how to do the decimation with meshlab

In [5]:
script_name = "decimation_meshlab.mls"

input_file_base = "/notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/Decimation/107816118160698192_original"
output_file = input_file_base + "_decimated"
meshlab_script_path_and_name = str(pathlib.Path.cwd()) + "/" + script_name


meshlab_fix_manifold_path_specific_mls(input_path_and_filename=input_file_base + ".off",
                                                   output_path_and_filename=output_file + ".off",
                                                   meshlab_script=meshlab_script_path_and_name)

xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/Decimation/107816118160698192_original.off -o /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/Decimation/107816118160698192_original_decimated.off -s /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/Decimation/decimation_meshlab.mls


'/notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/Decimation/107816118160698192_original_decimated.off'

# Testing theh boolean difference

In [ ]:
from pykdtree.kdtree import KDTree
import time
import trimesh
import numpy as np

def filter_mesh_significant_outside_pieces(unfiltered_mesh,main_mesh,
                                           significance_threshold=2000,
                                           n_sample_points=3,
                                          print_flag=False):
    """
    Purpose; will take in a full, unfiltered mesh and find the biggest mesh piece, and then return a list of that mesh 
    with all of the other mesh fragments that are both above the significance_threshold AND outside of the biggest mesh piece

    Pseudocode: 
    1) split the meshes to unconnected pieces
    2) Filter the meshes for only those above the significance_threshold
    3) find the biggest mesh piece
    4) Iterate through all of the remaining pieces:
        a. Determine if mesh inside or outside main mesh
        b. If outside add to final list to return

    Returns: 
    1) list of significant mesh pieces that are not inside of main mesh

    """
    if print_flag:
        print("------Starting the mesh filter for significant outside pieces-------")

    mesh_pieces = unfiltered_mesh.split(only_watertight=False)
    
    if print_flag:
        print(f"There were {len(mesh_pieces)} pieces after mesh split")

    significant_pieces = [m for m in mesh_pieces if len(m.faces) > significance_threshold]

    if print_flag:
        print(f"There were {len(significant_pieces)} pieces found after size threshold")
    if len(significant_pieces) <=0:
        print("THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold")
        return []



    final_mesh_pieces = []

    #final_mesh_pieces.append(main_mesh)
    for i,mesh in enumerate(significant_pieces):
        #get a random sample of points
        # points = np.array(mesh.vertices[:n_sample_points,:]) # OLD WAY OF DOING THIS
        idx = np.random.randint(len(mesh.vertices), size=n_sample_points)
        points = mesh.vertices[idx,:]


        start_time = time.time()
        signed_distance = trimesh.proximity.signed_distance(main_mesh,points)
        #print(f"Total time = {time.time() - start_time}")

        outside_percentage = sum(signed_distance <= 0)/n_sample_points
        if outside_percentage > 0.9:
            final_mesh_pieces.append(mesh)
            #print(f"Mesh piece {i} OUTSIDE mesh")
        else:
            #print(f"Mesh piece {i} inside mesh :( ")
            pass
                
    return final_mesh_pieces
    
def neuron_boolean_difference(main_mesh_verts,
                             main_mesh_faces,
                             child_mesh_verts,
                             child_mesh_faces,
                             distance_threshold=5,
                             significance_threshold=1000,
                             n_sample_points=3,
                             print_flag=False):
    """
    returns the boolean difference of two meshes passed. Only returns meshes pieces that 
    are greater than the size threshold and outside of the main mesh
    
    Function operation: child_mesh - main_mesh
  

    Parameters: 
    main_mesh_verts (np.array): array of the reference mesh vertices
    main_mesh_faces (np.array): array of the reference mesh faces
    child_mesh_verts (np.array): array of the child mesh vertices that is to be compared to the main mesh
    child_mesh_faces (np.array): array of the child mesh faces that is to be compared to the main mesh
    
    Optional parameters:
    -- for controlling the mesh returned --
    distance_threshold (int) : distance away from the reference mesh that vertices from the child
                                mesh will considered distinct and part of the difference mesh 
                                (default=5)
    significance_threshold (int) : number of faces necessary for any distinct/unconnected part of the 
                                    difference mesh to be considered relevant and included in the difference mesh
                                (default=1000)
    n_sample_points (int) : number of vertices to check to see if submesh is a mesh located inside of the main mesh.
                            The less there are the quicker the speed for filtering the difference mesh
                            (default=3)
    
    
    Returns:
    difference_mesh_verts (np.array): array of vertices from the mesh boolean operation of child - main mesh
    difference_mesh_faces (np.array): array of faces from the mesh boolean operation of child - main mesh
    """
    
    #Create the kdtree from main mesh and run the queries
    
    import time
    global_time = time.time()
    start_time = time.time()
    mesh_tree = KDTree(main_mesh_verts)
    distances,closest_node = mesh_tree.query(child_mesh_verts)
    if print_flag:
        print(f"Total time for KDTree creation and queries: {time.time() - start_time}")
    
    if print_flag:
        print("Original number vertices in child mesh = " + str(len(child_mesh_verts)))
    vertex_indices = np.where(distances > distance_threshold)[0]
    if print_flag:
        print("Number of vertices after distance threshold applied =  " + str(len(vertex_indices)))
    
    start_time = time.time()
    set_vertex_indices = set(list(vertex_indices))
    face_indices_lookup = np.linspace(0,len(child_mesh_faces)-1,len(child_mesh_faces)).astype('int')
    face_indices_lookup_bool = [len(set_vertex_indices.intersection(set(tri))) > 0 for tri in child_mesh_faces]
    face_indices_lookup = face_indices_lookup[face_indices_lookup_bool]

    if print_flag:
        print(f"Total time for finding faces after distance threshold applied: {time.time() - start_time}")
    
    start_time = time.time()
    trimesh_original = trimesh.Trimesh(child_mesh_verts,child_mesh_faces,process=False) 
    new_submesh = trimesh_original.submesh([face_indices_lookup],only_watertight=False,append=True)
    
    pymesh_mesh = trimesh.Trimesh(main_mesh_verts,main_mesh_faces)
    #return new_submesh
    #filter the mesh for only significant pieces on the outside
    returned_mesh = filter_mesh_significant_outside_pieces(new_submesh,pymesh_mesh,
                                                           significance_threshold,
                                                           n_sample_points=n_sample_points,
                                                          print_flag=print_flag)
    
    total_returned_mesh = trimesh.Trimesh(vertices = np.array([]),
                                     faces = np.array([]))
    for r in returned_mesh:
        total_returned_mesh = total_returned_mesh + r

    if print_flag:
        print(f"Total time for filtering: {time.time() - start_time}")
    
    if print_flag:
        print(f"Total time for boolean difference: {time.time() - global_time}")
    return total_returned_mesh.vertices,total_returned_mesh.faces

    
    

In [ ]:
largest_mesh = trimesh.load_mesh("temp/96631955273149705_decimated_poisson_largest_piece.off")
original_mesh = trimesh.load_mesh("temp/neuron_96631955273149705_decimated_bac.off")

mesh_pieces =neuron_boolean_difference( largest_mesh.vertices,
                                               largest_mesh.faces,
                                                original_mesh.vertices,
                                               original_mesh.faces,
                                       
                                               distance_threshold=1000,
                                               significance_threshold=1000,
                                               n_sample_points=10)

significant_leftovers = trimesh.Trimesh(vertices = mesh_pieces[0],
                                        faces = mesh_pieces[1]
                                       )

significant_leftovers.export("temp/significant_leftovers.off")dd